In [6]:
import argparse
import datetime
import pytz
import cx_Oracle as ora

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [11]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
#VIEW_ID в main()
# Путь к файлу client_secrets.json.
CLIENT_SECRETS_PATH = "c:\\SKP\\snq-scripts\\GooleAnalytics\\client_secret_878046399105-v070ud06bdjgmcmklf4ooila4j2qsdi5.apps.googleusercontent.com.json"

In [7]:
local_tz = pytz.timezone('Europe/Moscow')
def utc_to_local(utc_dt):
    local_dt = utc_dt.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return local_tz.normalize(local_dt) # .normalize might be unnecessary

In [8]:
def initialize_analyticsreporting():
  """Инициализирует объект службы analyticsreporting.

  Возвращает Analytics – авторизованный объект службы Analytics Reporting.
  """
  # Синтаксический анализ аргументов командной строки.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Настройка объекта Flow, который будет использоваться при необходимости аутентификации.
  flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))

  # Подготовьте учетные данные и авторизуйте с их помощью объект HTTP.
  # Если учетные данные недействительны или не существуют, воспользуйтесь оригинальной
  # процедурой клиента. Применение объекта Storage гарантирует, что в случае успеха правильные
  # учетные данные будут записаны обратно в файл.
  storage = file.Storage('analyticsreporting.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Создание объекта службы.
  analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)

  return analytics

In [9]:
def get_last_sql_data():
    #Oracle connection
    dsnStr = ora.makedsn("dwh.snq.ru", "1521", "DWH")
    con = ora.connect(user="ROMAN.BYCHKOV", password="communism88", dsn=dsnStr)

    #Take last date from SQL Table
    cur = con.cursor()
    cur.execute('''select max(dt) dt
                   from inet_traffic2 t''')
    for row in cur:
        max_data = row[0]
        max_data = utc_to_local(max_data)
        print("last date in inet_traffic2 {}".format(max_data))

    next_date = max_data + datetime.timedelta(days=1) #getting next date
    next_date_str = next_date.strftime("%Y-%m-%d") #next date to string format

    cur.close()
    con.close()

    return(next_date_str)

In [ ]:
analytics = initialize_analyticsreporting()